In [1]:
from TwitterAPI import TwitterAPI
from tqdm import tqdm
import pandas as pd
import os

In [10]:
keys = dict(consumer_key=os.environ['TWITTER_KEY'],
            consumer_secret=os.environ['TWITTER_SECRET'],
            access_token_key=os.environ['TWITTER_ACCESS_TOKEN_KEY'],
            access_token_secret=os.environ['TWITTER_ACCESS_TOKEN_SECRET'])
api = TwitterAPI(**keys)

In [28]:
ids = pd.read_csv('../data/list_ids.csv', index_col=0)
info_with_twitter = pd.read_csv('../data/congressperson_all_info.csv',
                                index_col=0)
info_with_twitter['state'] = info_with_twitter['state'].str.lower()

# Populate lists with users
We'll split up the congresspeople list by state and populate each list with it

In [32]:
# Now populate the lists with users
for state, i_id, i_url in tqdm(ids[['State', 'ID', 'URL']].values):
    # Pull values for this state
    values = info_with_twitter.query('state == @state')

    # Now post the data
    screen_names = ','.join(values.dropna(subset=['twitter'])['twitter'].values)
    resp = api.request('lists/members/create_all',
                       params={'list_id': i_id, 'screen_name': screen_names})
    if resp.status_code != 200:
        print('Failed to populate list')
        break

100%|██████████| 50/50 [00:20<00:00,  2.93it/s]


# Helper code

In [19]:
# Remove all users from lists
n_removed = []
for _, (state, i_link, i_uri, i_id) in tqdm(ids.iterrows()):
    resp = api.request('lists/members', params={'list_id': i_id, 'count': 1000})
    user_ids = [user['id_str'] for user in resp.json()['users']]
    user_string = ','.join(user_ids)
    resp = api.request('lists/members/destroy_all',
                       params={'list_id': i_id, 'user_id': user_ids})
    n_removed.append(len(user_ids))

50it [00:38,  1.74it/s]


In [ ]:
# Add a single member to a list
resp = api.request('lists/members/create',
                   params={'list_id': i_id, 'screen_name': 'cspan'})

resp.json()